<a href="https://colab.research.google.com/github/masadream/data_analytics/blob/master/Causal_Inference_studysession/Ch1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>